In [1]:
# ライブラリをインポート
from google.cloud import speech
import io
import os
from pydub import AudioSegment
import ffmpeg
import pandas as pd

# keyを指定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../Transcription/key/credentials.json'

In [2]:
#いきなり長いファイルは使用できないので、適当に30秒ほどのデータを作成するときに利用
sound = AudioSegment.from_file('../Transcription/audio/ステレオaudio.wav', format='wav')
print(len(sound))
# msで記述されている（5~1000秒の切り出し）
sound1 = sound[5000:59000]
sound1.export('output.wav', format='wav')

584640


<_io.BufferedRandom name='output.wav'>

In [14]:
# 文字起こし関数(モノラルチャンネルver)
def transcription_wav(audio_file, box, string_data):
    # 音声ファイルの読み込み
    with io.open(audio_file, 'rb') as f:
        content = f.read()

    # APIパラメータの作成
    audio = speech.RecognitionAudio(content = content)
    config = speech.RecognitionConfig(
        # 都度エンコーディングする場合は、LINEAR16 しない場合は、ENCODING_UNSPECIFIED
        # hertzはwavファイルによっては変更する必要があるかもしれないです。自分の場合は32000に指定しろと言われました
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 8000,
        language_code = "ja-JP",
        #enable_word_time_offsets=True
    )

    # APIの呼び出し
    client = speech.SpeechClient()
    response = client.recognize(config = config, audio = audio)

    # 結果の表示
    # for result in response.results:
    #     print(result.alternatives[0].transcript)
    for i, result in enumerate(response.results):
        # print(result)
        # print(i)
        alternative = result.alternatives[0]
        print('-'*20)
        print('現在文字起こし中です')
        # print('first alternative of result {}'.format(i))
        # print(u'Transcript:{}'.format(alternative.transcript))
        # print(u'Channel Tag:{}'.format(result.channel_tag))
        # print('second is ===', result.result_end_time)
        item = [format(alternative.transcript), result.result_end_time, string_data]
        box.append(item)
    print('文字起こしを終了します')
    return box

In [7]:
from numpy import int16
import wave
import struct
import math
import numpy as np

# wavファイルを59秒間隔で分割する関数
def cut_wav(audio_chang_wav, time, wav_cut_dir):
    wr = wave.open(audio_chang_wav, "r")

    # wav情報を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr
    integer = math.floor(total_time)
    t = int(time)
    frames = int(ch * fr * t)
    # 小数点切り上げ（1分に満たない最後のシーンを出力するため）
    num_cut = int(math.ceil(integer / t))
    data = wr.readframes(wr.getnframes())
    wr.close()

    X = np.frombuffer(data, dtype=int16)

    for i in range(num_cut):
        outf = wav_cut_dir + str(i) + ".wav"
        start_cut = int(i * frames)
        end_cut = int(i * frames + frames)
        print(start_cut)
        print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, "w")
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()
        print('終了しました')

In [8]:
# APIに直接遅れるのが60秒未満なので、59秒で設定している
# 1分以上の場合は、uriでの扱いになる？
time = 59
audio_change_wav = '../Transcription/audio/ステレオaudio_change.wav'
wav_cut_dir = '../Transcription/test_file/'
# 音声ファイルをカットする関数
cut_wav(audio_change_wav,time,  wav_cut_dir)


0
944000
終了しました
944000
1888000
終了しました
1888000
2832000
終了しました
2832000
3776000
終了しました
3776000
4720000
終了しました
4720000
5664000
終了しました
5664000
6608000
終了しました
6608000
7552000
終了しました
7552000
8496000
終了しました
8496000
9440000
終了しました


In [ ]:
import soundfile
# いただいたファイルの音声ファイルのbit数が8bitだったので変換時に16bitに変換しろとエラーが出た
# bit数を変換する関数
def bit_change(audio_file_path, subtype):
    data, fs = soundfile.read(audio_file_path)
    soundfile.write('../Transcription/audio/ステレオaudio_change.wav', data, fs, subtype=subtype)
    print('bit数の変換ファイルの出力が終了しました。')

In [ ]:
audio_path = '../Transcription/audio/ステレオaudio.wav'
change_subtype = 'PCM_16'
# bit数を変換するための関数実行
bit_change(audio_path,change_subtype)

In [13]:
# ステレオチャンネルの音声の文字起こしをする関数
def transcription_wav_stereo(audio_file, box, string_data):
    client = speech.SpeechClient()

    with io.open(audio_file, 'rb') as f:
        content = f.read()

    audio = speech.RecognitionAudio(content=content)

    config = speech.RecognitionConfig(
        encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz = 8000,
        language_code = 'ja-JP',
        audio_channel_count = 2,
        enable_separate_recognition_per_channel = True,
        # 機械学習モデルを選択できるので、それで電話通話を選択する。
        model="phone_call",
    )

    response = client.recognize(config=config, audio=audio)

    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        print('-'*20)
        print('現在文字起こし中です')
        # print('first alternative of result {}'.format(i))
        # print(u'Transcript:{}'.format(alternative.transcript))
        # print(u'Channel Tag:{}'.format(result.channel_tag))
        # print('second is ===', result.result_end_time)
        item = [alternative.transcript, result.channel_tag, result.result_end_time, string_data]
        box.append(item)

    print('文字起こしを終了します。')
    return box


In [ ]:
from pydub import AudioSegment
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

# ステレオチャンネルの音声をleft, rightで分割して出力する関数
def stereo_channel_split(stereo_audio):
    sound = AudioSegment.from_file(stereo_audio)
    # チャンネル数
    channel_count = sound.channels
    # frame_rate = speech to text でいうところのhertz
    frames_per_second = sound.frame_rate
    # ファイルの音声の長さ（秒）
    duration = sound.duration_seconds
    # 音声ファイルをnumpyで変換
    sound_array = np.array(sound.get_array_of_samples())
    # 右左に分割
    left_sound = sound_array[0:len(sound_array):2]
    right_sound = sound_array[1:len(sound_array):2]
    # 音声データの書き出し
    write('left_audio.wav', frames_per_second, left_sound)
    write('right_audio.wav', frames_per_second, right_sound)
    print('音声の分割が終了しました')

# 試しに実行
audio_path = '../Transcription/audio/ステレオaudio_change.wav'
stereo_channel_split(audio_path)

In [15]:
test_data_box = []
for i in range(0, 2):
    #transcription_wav('../Transcription/test_file/'+str(i)+'.wav', test_data_box, 'audio'+str(i))
    transcription_wav_stereo('../Transcription/test_file/'+str(i)+'.wav', test_data_box, 'audio'+str(i))
    print(str(i)+'個目のデータの文字起こしが終了しました')


# 音声を分割した時のために利用する方
#df = pd.DataFrame(test_data_box, columns=['text', 'timestamp', 'audio_num'])
# ステレオチャンネルでそのまま利用した場合に利用する方
df = pd.DataFrame(test_data_box, columns=['text', 'tag', 'timestamp', 'audio_num'])
#df.to_csv('test_stereo.csv')


--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
文字起こしを終了します。
0個目のデータの文字起こしが終了しました
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
--------------------
現在文字起こし中です
文字起こしを終了します。
1個目のデータの文字起こしが終了しました
